In [5]:
from sympy.abc import _clash2
from sympy import symbols

In [82]:
str(symbols('beta_1'))

'beta_1'

In [238]:
def symbol_prod(*args):
    output = args[0]*args[1]
    for i, arg in enumerate(args):
        if i <= 1:
            pass
        else:
            output *= arg
    return output


@dataclass
class sample_generator:
    var_num: int
    sample_size: int
    num_interactions: int = None
    rng: np.random._generator.Generator = np.random.default_rng()
    beta_range: Tuple[Union[float]] = (3, 8)
    pi_range : Tuple[Union[float]] = (.2, .8)
    error_scale : float = sqrt(5)


    def gen_main_effects(self):
        self._main_effects_variabls = [Symbol(f'x_{i+1}') for i in range(self.var_num)]
        self._main_beta = [Symbol(f'beta_{i+1}') for i in range(self.var_num)]
    
    @property
    def main_effect_coefficients(self):
        if hasattr(self, '_main_beta'):
            pass
        else:
            self.gen_main_effects()
        return self._main_beta
    
    @property
    def main_effect_variables(self):
        if hasattr(self,'_main_effects_variables'):
            pass
        else:
            self.gen_main_effects()
        return self._main_effects_variabls
    
    def interaction_variables_generator(self):
        i = 1
        for k in range(2, self.var_num + 1):
            for idx, var_name in zip(range(i, i+comb(self.var_num, k)), combinations(self.main_effect_variables, k)):
                yield idx, symbol_prod(*var_name)
            i += comb(self.var_num, k)

    def gen_interactions(self):
        main_variables = self.main_effect_variables
        num_interactions_total = 2**self.var_num - 1 - self.var_num
        # setting the number of interactions
        if self.num_interactions:
            if (self.num_interactions >= 0) and (self.num_interactions <= num_interactions_total):
                pass
            else:
                self.num_interactions = int(self.rng.uniform(1, num_interactions_total))
        else:
            self.num_interactions = int(self.rng.uniform(1, num_interactions_total))

        chosen_index = np.sort(self.rng.choice(range(1, num_interactions_total+1), self.num_interactions, replace = False))
        chosen_interactions = [var_name for idx, var_name in self.interaction_variables_generator() if idx in chosen_index.tolist()]
        chosen_index = [1 for _ in range(self.var_num + 1)] + \
        [1 if x in chosen_index.tolist() else 0 for x in range(1, num_interactions_total+ 1)]
        self.interactions_coef = [x if self.rng.random() < .5 else -x for x \
                                    in self.rng.uniform(low=self.beta_range[0], high=self.beta_range[1], size=self.num_interactions)]
        self.beta_effective = chosen_index
        self._interactions = dict(zip(chosen_interactions, self.interactions_coef))
    
    @property
    def interaction_effect_coefficients(self):
        if hasattr(self, '_interactions'):
            pass
        else:
            self.gen_interactions()
        return self._interactions
    
    @property
    def main_coefficients(self):
        if hasattr(self, '_main_coefficients'):
            pass
        else:
            self._main_coefficients = [x if self.rng.random() < .5 else -x for x in self.rng.uniform(low=self.beta_range[0], high=self.beta_range[1], size=self.var_num+1)]
        return {Symbol(f'beta_{i}'): v for i,v in enumerate(self._main_coefficients)}
    

    ## generating X and y
    @property
    def pi(self):
        if hasattr(self, '_pi'):
            pass
        else:
            self._pi = self.rng.uniform(low = self.pi_range[0], high = self.pi_range[1], size = self.var_num)
        return {f'pi_{i+1}': v for i,v in enumerate(self._pi)}
    
    def X_generator(self):
        for _ in range(self.sample_size):
            yield bernoulli.rvs(list(self.pi.values()), size = self.var_num)

    def get_y(self, x):
        main_effect_variables = self.main_effect_variables
        main_effect_coefficients = self.main_coefficients
        interaction_effect_coefficients = self.interaction_effect_coefficients




    @property
    def y(self):
        if hasattr(self, '_y'):
            pass
        else:
            beta = self.beta
            interactions = self.interactions
            y = []
            
            def find_interaction_effect(row, interactions = self.interactions):
                output = 0
                for key, val in interactions.items():
                    output += row[key.split('*')].prod() * val
                return output
            
            
            for idx, row in self.X.iterrows():
                main = beta['beta_0'] + sum([x*y for x,y in zip(list(beta.values())[1:], row)])
                interaction_eff = find_interaction_effect(row)
                error = norm.rvs(scale = self.error_scale)
                y.append(main + interaction_eff + error)
            
            self._y = np.array(y).reshape(-1,1)
        return self._y
        

In [218]:
x = sample_generator(3, 10)
gen = x.X_generator()

In [219]:
x.interaction_effect_coefficients

{x_1*x_2: 5.452370076923513,
 x_2*x_3: -3.743476020443744,
 x_1*x_2*x_3: 5.334953848094771}

In [224]:
series = pd.Series(next(gen), index = ['x_1','x_2','x_3'])

In [231]:
key_1 = list(x.interaction_effect_coefficients.keys())[0]

In [236]:
series[str(key_1).split("*")].prod()

0

In [237]:
x.main_effect_variables

[x_1, x_2, x_3]

In [220]:
x.main_coefficients

{beta_0: -3.8256511538698827,
 beta_1: 3.6753237315622074,
 beta_2: -5.579813024370846,
 beta_3: -5.473873890923027}

In [8]:
from dataclasses import dataclass
from typing import List, Union, Tuple
import numpy as np
from scipy.stats import bernoulli, norm
import pandas as pd
from functools import partial
from itertools import combinations, product
from math import comb, sqrt
from sympy import symbols, Symbol

@dataclass
class sample_generator:
    p: int
    sample_size: int
    num_interactions: int = None
    rng: np.random._generator.Generator = np.random.default_rng()
    beta_range: Tuple[Union[int, float]] = (3, 8)
    pi_range: Tuple[Union[float]] = (.2, .8)
    error_scale: float = sqrt(5)
        
    @property
    def interactions(self):
        if hasattr(self, '_interactions'):
            pass
        else:
            beta = [f'beta_{i}' for i in range(1, self.p+1)]
            beta_interaction_coef = []
            if self.num_interactions:
                assert self.num_interactions <= 2**self.p -1 - self.p
            else:
                self.num_interactions = int(self.rng.uniform(0, 2**self.p - 1 - self.p))
            r = self.num_interactions 
            size_total_int = 2**self.p -1 - self.p
            chosen_index = np.sort(self.rng.choice(range(size_total_int), r, replace = False))
            chosen_interactions = []
            i = 0
            for k in range(2, self.p + 1):
                chosen_interactions += ['*'.join(beta_name) for idx, beta_name in \
                                        zip(range(i, i+comb(self.p, k)), combinations(beta, k)) \
                                        if idx in chosen_index.tolist() ]
                i += comb(self.p, k)
            chosen_index = [1 for _ in range(self.p + 1)] + \
            [1 if x in chosen_index.tolist() else 0 for x in range(size_total_int)]
            self.interactions_coef = [x if self.rng.random() < .5 else -x for x \
                                      in self.rng.uniform(low=self.beta_range[0], high=self.beta_range[1], size=r)]
            self.beta_effective = chosen_index
            self._interactions = {x.replace('beta_', 'X'):val for x, val in zip(chosen_interactions, self.interactions_coef)}
        return self._interactions
        
    @property
    def beta(self):
        if hasattr(self, '_beta'):
            pass
        else:
            self._beta = [x if self.rng.random() < .5 else -x for x in self.rng.uniform(low=self.beta_range[0], high=self.beta_range[1], size=self.p+1)]
        return {f'beta_{i}': v for i,v in enumerate(self._beta)}
    
    @property
    def beta_names(self):
        if hasattr(self, '_beta_names'):
            pass
        else:
            beta_raw_names = [f'beta{i}' for i in range(1, self.p + 1)]
            beta_names = [f'beta{i}' for i in range(self.p + 1)]
            for r in range(2, self.p):
                beta_names += ['*'.join(x) for x in combinations(beta_raw_names, r)]
            beta_names += ['*'.join(beta_raw_names)]
            self._beta_names = beta_names
            del beta_names
            del beta_raw_names
        return self._beta_names
    
    @property
    def pi(self):
        if hasattr(self, '_pi'):
            pass
        else:
            self._pi = self.rng.uniform(low = self.pi_range[0], high = self.pi_range[1], size = self.p)
        return {f'pi_{i+1}': v for i,v in enumerate(self._pi)}
    
    @property
    def X(self):
        if hasattr(self, '_X'):
            pass
        else:
            self._X = pd.DataFrame({f'X{i+1}':self.x(pi, self.sample_size) for i, pi in enumerate(self.pi.values())}).astype(np.ubyte)
        return self._X
        
        
    @property
    def y(self):
        if hasattr(self, '_y'):
            pass
        else:
            beta = self.beta
            interactions = self.interactions
            y = []
            
            def find_interaction_effect(row, interactions = self.interactions):
                output = 0
                for key, val in interactions.items():
                    output += row[key.split('*')].prod() * val
                return output
            
            
            for idx, row in self.X.iterrows():
                main = beta['beta_0'] + sum([x*y for x,y in zip(list(beta.values())[1:], row)])
                interaction_eff = find_interaction_effect(row)
                error = norm.rvs(scale = self.error_scale)
                y.append(main + interaction_eff + error)
            
            self._y = np.array(y).reshape(-1,1)
        return self._y
            
            
        
# for idx, row in rng.X.iterrows():
#     print(rng.beta['beta_0'] + sum([x*y for x,y in zip(list(rng.beta.values())[1:], row)]))        
        
        
        
    @property
    def barcode(self):
        if hasattr(self, '_barcode'):
            pass
        else:
            self._barcode = self.gen_barcode(self.X)
        return self._barcode
    
    @staticmethod
    def gen_barcode(X):
        pack_bits = np.packbits(np.array(X), axis = -1)
        if pack_bits.shape[1]>1:
            def gen_barcode(a, k, return_float = False):
                m = len(a)-1
                total_sum = 0
                for i, x in enumerate(a):
                    if i < m:
                        if return_float:
                            adjust = (x * 2**(m-i-1) * 2**k)
                            total_sum += float(adjust)
                        else:
                            total_sum += (x * 2**(m-i-1) << k)
                    else:
                        total_sum += (x >> (8-k))
                return total_sum
            if X.shape[1] > 500:
                barcode = partial(gen_barcode, k = X.shape[1]%8, return_float = True)
            else:
                barcode = partial(gen_barcode, k = X.shape[1]%8)
        else:
            def adjust_barcode(a, k):
                a = a >> k
                return a
            barcode = partial(adjust_barcode, k = 8-X.shape[1])
        output = np.apply_along_axis(barcode, 1, pack_bits)
        return output

    @staticmethod
    def barcode_to_beta(barcode):
        if isinstance(barcode, list):
            output = [1] + barcode
        else:
            output = [1] + list(barcode)
        N = len(output)
        for i in range(2, N):
            output += [np.prod(x) for x in combinations(barcode, i)]
    #     output += [np.prod(output)]
        return output
    
    @property
    def L(self):
        if hasattr(self, '_L'):
            pass
        else:
            all_sets = list(set(product([0,1], repeat = self.p))); all_sets.sort()
            self._L = np.array([self.barcode_to_beta(x) for x in all_sets]).T
        return self._L